In [1]:
!pip install faiss-gpu
!pip install sentence_transformers
!pip install einops
!pip install --upgrade transformers sentence-transformers
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 71.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [2]:
from sentence_transformers import SentenceTransformer
import gdown
import faiss
import pandas as pd
import re
import numpy as np
import json

In [3]:
model_bge = SentenceTransformer("BAAI/bge-m3")
model_e5 = SentenceTransformer("intfloat/multilingual-e5-large")
model_tuned = SentenceTransformer("Quintu/bge-m3-legal_retrieval")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [4]:
# load faiss index
index_bge = faiss.read_index('/kaggle/input/bin-pretrained/bge_m3.bin')
index_e5 = faiss.read_index('/kaggle/input/bin-pretrained/e5.bin')
index_tuned = faiss.read_index('/kaggle/input/bge-m3-23400/bge_m3_23400.bin')


In [5]:
data = pd.read_csv('/kaggle/input/val-data/val_data.csv') 
questions = data['question'].tolist()
qids = data['qid'].tolist()


In [6]:
def process_cid(s):
    s = s.strip('[]') 
    elements = re.split(r'[\,\s]+', s) 
    return [int(element) for element in elements if element.strip().isdigit()]  

result = {}

for idx, row in data.iterrows():
    qid = row['qid']
    cid = process_cid(row['cid'])
    result[qid] = cid
    

In [7]:
# encode question
embeddings_bge = model_bge.encode(questions)
embeddings_e5 = model_e5.encode(questions)
embeddings_tuned = model_tuned.encode(questions)

Batches:   0%|          | 0/744 [00:00<?, ?it/s]

Batches:   0%|          | 0/744 [00:00<?, ?it/s]

Batches:   0%|          | 0/744 [00:00<?, ?it/s]

In [8]:
def search_faiss(index, query_embedding, k=1500):
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)  
    return distances, indices

In [9]:
distances_bge, indices_bge = search_faiss(index_bge, embeddings_bge)
distances_e5, indices_e5 = search_faiss(index_e5, embeddings_e5)
distance_tuned, indeces_tuned = search_faiss(index_tuned, embeddings_tuned)

In [10]:
npy_data = np.load('/kaggle/input/bm25-search/val_search_bm25.npy', allow_pickle=True)

In [49]:
weight_bge = 1
weight_e5 = 0

all_id_with_score = []
top_k = 100

for i in range(len(indices_bge)):
    bge_top_ids = indices_bge[i][:top_k]
    bge_top_scores = distances_bge[i][:top_k]
    e5_top_ids = indices_e5[i][:top_k]
    e5_top_scores = distances_e5[i][:top_k]
    bge_tuned_top_ids = indeces_tuned[i][:top_k]
    bge_tuned_top_scores = distance_tuned[i][:top_k]
    
    combined_scores = {}

    for idx, score in zip(bge_tuned_top_ids, bge_tuned_top_scores):
        if idx not in combined_scores:
            combined_scores[idx] = 0
        combined_scores[idx] += weight_bge * score

    for idx, score in zip(e5_top_ids, e5_top_scores):
        if idx not in combined_scores:
            combined_scores[idx] = 0
        combined_scores[idx] += weight_e5 * score

    cid_score_data = npy_data[i]

    for cid, npy_score in cid_score_data:
        if cid in combined_scores:
            combined_scores[cid] *= npy_score 
    sorted_combined = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    top_100_ids = [item[0] for item in sorted_combined[:100]]
    all_id_with_score.append(top_100_ids)


with open('result_bi_encoder.txt', 'w') as f:
    for qid, ids in zip(qids, all_id_with_score):
        ids_str = ' '.join(map(str, ids))
        f.write(f"{qid} {ids_str}\n")
print("complete")


complete


In [50]:
import os
import zipfile

output_folder = "bge-bm25+bm25"
zip_filename = f"{output_folder}.zip"
os.makedirs(output_folder, exist_ok=True)

num_parts = 9
lines_per_part = len(qids) // num_parts

for part in range(num_parts):
    file_path = os.path.join(output_folder, f'output_part_{part + 1}.txt')
    with open(file_path, 'w') as f:
        start_index = part * lines_per_part
        end_index = (part + 1) * lines_per_part if part != num_parts - 1 else len(qids)
        for qid, ids in zip(qids[start_index:end_index], all_id_with_score[start_index:end_index]):
            ids_str = ' '.join(map(str, ids))
            f.write(f"{qid} {ids_str}\n")

# Nén thư mục thành một file ZIP

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_folder))

print(f"All files are saved in '{output_folder}' and zipped as '{zip_filename}'.")


All files are saved in 'bge-bm25+bm25' and zipped as 'bge-bm25+bm25.zip'.


In [51]:
with open('/kaggle/input/template/template_chunking_flat.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
stt_to_infor_id = {item['stt']: item['infor_id'] for item in json_data}


In [52]:


with open('/kaggle/working/result_bi_encoder.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('result_bi_encoder_convert.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        if not line.strip():
            continue
        parts = line.strip().split()
        qid = parts[0]
        updated_ids = [stt_to_infor_id.get(int(stt), stt) for stt in parts[1:]]  
        ids_str = ' '.join(map(str, updated_ids))
        f.write(f"{qid} {ids_str}\n")

print("File đã được lưu với infor_id thay vì stt.")


File đã được lưu với infor_id thay vì stt.


In [53]:
import pandas as pd


def convert_to_list(s):
    s = s.strip('[]') 
    elements = s.split() 
    return [int(element) for element in elements]

def read_csv(file_path):
    df = pd.read_csv(file_path)
    df['cid'] = df['cid'].apply(convert_to_list)
    return df

def read_txt(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [list(map(int, line.split())) for line in lines]

def calculate_mrr(submit, answer):
    mrr_scores = []
    for result in submit:
        qid = result[0]  
        cids = result[1:11]  
        answer_cids = answer.get(qid, [])
        rank = None
        for i, cid in enumerate(cids):
            if cid in answer_cids:  
                rank = i + 1  
                break
        if rank: mrr_scores.append(1 / rank)
    return sum(mrr_scores) / len(mrr_scores) if mrr_scores else 0

answer_df = read_csv('/kaggle/input/val-data/val_data.csv') 
submit_data = read_txt('/kaggle/working/result_bi_encoder_convert.txt')  

answer_dict = {row['qid']: row['cid'] for _, row in answer_df.iterrows()}

mrr_score = calculate_mrr(submit_data, answer_dict)
print(f"MRR@10: {mrr_score}")


MRR@10: 0.7421349192439479
